<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/GS%20-%20Guardar%20Vocabulario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [12]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/base-diputados -f base_formateada_snowball.csv
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f partidos.csv

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
 99% 211M/214M [00:03<00:00, 86.8MB/s]
100% 214M/214M [00:03<00:00, 73.9MB/s]
  0% 0.00/2.55k [00:00<?, ?B/s]
100% 2.55k/2.55k [00:00<00:00, 5.90MB/s]


In [13]:
import pandas as pd
base=pd.read_csv('base_formateada_snowball.csv')

<ipython-input-13-9ac41a69d66e>:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('base_formateada_snowball.csv')


In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])
stopwords.extend(['podemosen', 'podemen', 'ministro', 'ministra', 'ss', 'ruego', 'aprobado', 'aprobada'])
stopwords.extend(['diputado', 'diputada', 'don', 'doña', 'vicepresidenta', 'sagastizab', 'unzetabarrenetxea', 'agirretxea', 'urresti', 'escrutinio', 'dijo', 'dice', 'quevedo'])
stopwords.extend(['campuzano', 'beitialarrangoitia', 'gonzalezmoro', 'mixto'])

In [16]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.namey.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))
nombres.remove('')

In [17]:
stopwords.extend(nombres)

In [19]:
partidos=pd.read_csv('partidos.csv', encoding='latin1', sep=';')

In [20]:
for x in partidos['nombre'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(str(a).split())
  stopwords.extend(a)

for x in partidos['1'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

for x in partidos['2'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

for x in partidos['3'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

In [21]:
len(stopwords)

4481

In [22]:
# elimino acentos de los tokens
a,b = 'áàéèíóúü','aaeeiouu'
trans = str.maketrans(a,b)
lista = []
for row in base.values:
    row[20]=row[20].translate(trans)
    lista.append(row)

base = pd.DataFrame(lista, columns=base.columns)

In [23]:
# llevo las stopwords a minúsculas y elimino acentos
stopwords=[x.lower() for x in stopwords if type(x)==str]
stopwords=[x.translate(trans) for x in stopwords if type(x)==str]

# Mencionadas en al menos 10 discursos
(spoken in at least 10 unique speaker-session)

In [24]:
base['tokens']

0         ['señor', 'señor', 'diput', 'abre', 'junt', 'p...
1         ['frail', 'poujad', 'segun', 'relacion', 'cons...
2         ['sigu', 'rueg', 'esperab', 'arteag', 'tom', '...
3         ['reanud', 'proced', 'dar', 'lectur', 'relacio...
4         ['leid', 'inform', 'junt', 'electoral', 'centr...
                                ...                        
334416                                         ['convalid']
334417    ['final', 'noviembr', 'adopt', 'med', 'urgent'...
334418          ['convalid', 'muchisim', 'señor', 'levant']
334419    ['buen', 'tard', 'señor', 'abre', 'comenz', 'c...
334420    ['continu', 'relat', 'deleg', 'president', 'ca...
Name: tokens, Length: 334421, dtype: object

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords , min_df=10)
bow = vect.fit_transform(base['tokens'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahuja', 'alcañiz', 'amor', 'arreciado', 'barcena', 'barnuevo', 'capel', 'capitan', 'cascos', 'castejon', 'cienfuegos', 'compromis', 'congr', 'escandon', 'estefani', 'figueroa', 'fortun', 'hickman', 'huarte', 'jeltzalea', 'jovellanos', 'lasquetty', 'latorre', 'legaz', 'lersundi', 'llin', 'machuca', 'maillo', 'mancisidor', 'margallo', 'maribona', 'marlaska', 'medel', 'menchero', 'mendicoa', 'na', 'olivares', 'patxi', 'peces', 'pedrero', 'pinilla', 'piñero', 'ponga', 'pujalte', 'salmones', 'santesmases', 'sito', 'soro', 'sotelo', 'tagle', 'tesorero', 'tizon', 'toledano', 'torrecilla', 'trevijano', 'valdecasas', 'valdes', 'vargas', 'villaseñor', 'ximo', 'yañez'] not in stop_words.
  warnings.warn(


In [26]:
total_features = len(vect.vocabulary_)

# Mencionadas al menos 100 veces en total
(spoken at least 100 times across al sessions)

In [27]:
new_voc={}
for token in vect.vocabulary_:
  if vect.vocabulary_[token]>100:
    new_voc[token]=vect.vocabulary_[token]

In [28]:
len(new_voc)

357817

# Mencionadas al menos 10 veces en un día
(spoken at least 10 times in at least one session)

In [29]:
base['fecha'] = pd.to_datetime(base['fecha'], format='%d/%m/%Y')

In [30]:
# Gentzkow et.al. usan mencionadas al menos 10 veces en una "sesión". Acá se usa mencionadas al menos 10 veces en un "día".

new_voc2={}
vect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords)
for x in base.fecha.unique():
  base_dia=base[pd.DatetimeIndex(base.fecha)==x]
  try:
    bow = vect.fit_transform(base_dia['tokens'])
    for token in vect.vocabulary_:
      if vect.vocabulary_[token]>10:
        new_voc2[token]=vect.vocabulary_[token]
  except: pass

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahuja', 'alcañiz', 'amor', 'arreciado', 'barcena', 'barnuevo', 'capel', 'capitan', 'cascos', 'castejon', 'cienfuegos', 'compromis', 'congr', 'escandon', 'estefani', 'figueroa', 'fortun', 'hickman', 'huarte', 'jeltzalea', 'jovellanos', 'lasquetty', 'latorre', 'legaz', 'lersundi', 'llin', 'machuca', 'maillo', 'mancisidor', 'margallo', 'maribona', 'marlaska', 'medel', 'menchero', 'mendicoa', 'na', 'olivares', 'patxi', 'peces', 'pedrero', 'pinilla', 'piñero', 'ponga', 'pujalte', 'salmones', 'santesmases', 'sito', 'soro', 'sotelo', 'tagle', 'tesorero', 'tizon', 'toledano', 'torrecilla', 'trevijano', 'valdecasas', 'valdes', 'vargas', 'villaseñor', 'ximo', 'yañez'] not in stop_words.
  warnings.warn(


In [31]:
len(new_voc2)

4704874

# Combinación

In [32]:
inter=set(new_voc2).intersection(new_voc)

In [33]:
vect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords ,vocabulary=inter)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

In [34]:
# La intersección genera un vocabulario de apenas 270230 bigramas.
total_features

357780

In [35]:
matrix = bow

In [38]:
import pickle

with open('vocabulary_snowball.voc','wb') as f:
   pickle.dump(inter, f)